In [ ]:
import argparse
import warnings
import os 
from scripts.infer_selection import test

os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
warnings.filterwarnings("ignore")   


In [ ]:
# Test the Slice Selction Model
#Input Scans - nrrd files in ../data/test/input/ folder
#Model - C3_Top_Selection_Model_Weight.hdf5 file /model/test
#Output - C3_Top_Slice_Prediction.csv' stored in ../data/test/output_csv/ folder
model=test('../data/test/input/','../model/test/C3_Top_Selection_Model_Weight.hdf5', '../data/test/output_csv/C3_Top_Slice_Prediction.csv')

In [ ]:
# Test the Segmentation Model
from scripts.infer_segmentation import test

#Input Scans - nrrd files in ../data/test/input/ folder
#Input csv - C3_Top_Slice_Prediction.csv' stored in ../data/test/output_csv/ folder
#Model - C3_Top_Segmentation_Model_Weight.hdf5 file /model/test
#Output - auto segementations stored in ../data/test/output_segmentation/ folder


model=test('../data/test/input','../model/test/C3_Top_Segmentation_Model_Weight.hdf5','../data/test/output_csv/C3_Top_Slice_Prediction.csv','../data/test/output_segmentation/')

In [ ]:
# calculate C3 muscle cross sectional area for each of the scans and store it in the output_csv folder
import pandas as pd
import os
from scripts.image_processing.image_window import get_image_path_by_id,apply_window
from scripts.image_processing.slice_array_from_nifty import get_C3_seg_array_by_id
import SimpleITK as sitk
import numpy as np
from pprint import pprint

csv_path = '../data/test/output_csv/C3_Top_Slice_Prediction.csv'
df_infer = pd.read_csv(csv_path, index_col = 0)

# for writing metrics into a CSV
df_init = pd.DataFrame()


img_dir  = '../data/test/input/'
auto_seg_dir ='../data/test/output_segmentation/'
csv_write_path = '../data/test/output_csv/C3_body_comp_area_density.csv'

for idx in range(df_infer.shape[0]):
    patient_id =df_infer.iloc[idx,1]
    c3_slice_auto = df_infer.iloc[idx,2]
    image_path =  get_image_path_by_id(patient_id, img_dir)
    auto_seg_path = get_image_path_by_id(patient_id, auto_seg_dir)
    
    if os.path.exists(image_path) and os.path.exists(auto_seg_path):

        muscle_auto_area,sfat_auto_area,vfat_auto_area = \
                            get_c3_slice_area(patient_id,c3_slice_auto,auto_seg_dir)  

        muscle_auto_density,sfat_auto_density,vfat_auto_density = \
                            get_c3_slice_density(patient_id,c3_slice_auto,auto_seg_dir,img_dir)

        round_num = 2
        
        #Data Frame rows for writing into a CSV File
        df_inter1 = pd.DataFrame({'patient_id':patient_id,
                                    'muscle_auto_segmentation_area':round(muscle_auto_area, round_num),
                                    'muscle_auto_sedensity':round(muscle_auto_density, round_num)
                                        },index=[0])

        df_init = df_init.append(df_inter1)
        df_init.to_csv(csv_write_path)
        print(idx,'th', patient_id, 'writen to', csv_write_path)

        print()
